<a href="https://colab.research.google.com/github/ymoslem/Sentence-Similarity/blob/main/Semantic_Search_Faiss_Multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilingual Semantic Search with Faiss

This notebook demonstrates how to use Faiss and Sentence-Transformers to perform multilingual semantic search. We have a query in English ("education and schools in Germany") and we retrieve top results from documents in the Ukrainian language.

* Faiss: https://github.com/facebookresearch/faiss
* Faiss tutorial: https://www.pinecone.io/learn/series/faiss/faiss-tutorial/
* Sentence-Transformers: https://github.com/UKPLab/sentence-transformers
* Sentence-Transformers documentation: https://sbert.net/

* Notebooks for some other similarity techniques [here](https://github.com/ymoslem/Sentence-Similarity).

In [ ]:
!pip3 install faiss-gpu sentence_transformers -q

In [ ]:
# Download files
!git clone https://github.com/ymoslem/Notion-Scraper.git -q

%cd Notion-Scraper/output/
!ls

/content/Notion-Scraper/output
Education	     Ірландія.json  Нідерланди.json  Угорщина.json
Австрія.json	     Ісландія.json  Німеччина.json   Україна.json
Аргентина.json	     Іспанія.json   Норвегія.json    Фінляндія.json
Бельгія.json	     Італія.json    Польща.json      Франція.json
Болгарія.json	     Канада.json    Португалія.json  Хорватія.json
Великобританія.json  Кіпр.json	    Румунія.json     Чехія.json
Греція.json	     Країна.json    Сербія.json      Чорногорія.json
Грузія.json	     Латвія.json    Словаччина.json  Швейцарія.json
Данія.json	     Литва.json     Словенія.json    Швеція.json
Естонія.json	     Мексика.json   США.json
Ізраїль.json	     Молдова.json   Туреччина.json


In [ ]:
import json
import os

work_dir = "."
json_files = [file_name for file_name in os.listdir(work_dir) if file_name.endswith(".json")]

corpus = []

for json_file in json_files:
  with open(os.path.join(work_dir,json_file)) as json_input:
    json_content = json.load(json_input)
    for item in json_content:
      url = item["url"]
      text_paragraphs = item["text"].split("\n")
      text_paragraphs = [(para, json_file[:-5], item["topic"], item["url"]) for para in text_paragraphs if len(para.split())>10 \
                     and (para, json_file[:-5], item["topic"], item["url"]) not in corpus]
      corpus += text_paragraphs

corpus[0:5]

[('Якщо ви постійно проживали в Україні і виїхали з країни, щоб уникнути війни з 24 лютого 2022 року, ви маєте право на тимчасовий захист відповідно до Виконавчого рішення Ради (ЄС) 2022/382 від 4 березня 2022 року, що встановлює наявність масового притоку переміщених осіб з України у розумінні статті 5 Директиви 2001/55/ЄС та має наслідком введення тимчасового захисту (Council Implementing Decision (EU) 2022/382 of 4 March 2022 establishing the existence of a mass influx of displaced persons from Ukraine within the meaning of Article 5 of Directive 2001/55/EC, and having the effect of introducing temporary protection).',
  'Кіпр',
  'Загальна інформація',
  'https://uahelpinfo.notion.site/4fee11cb7559421b8b2a8d60ab26950a'),
 ('Тимчасовий захист триватиме щонайменше один рік і може бути продовжений залежно від ситуації в Україні. Права згідно з Директивою про тимчасовий захист включають: дозвіл на проживання, доступ до ринку праці та житла, медичну допомогу та доступ до освіти для діте

In [ ]:
from sentence_transformers import SentenceTransformer

corpus_sentences = [item[0] for item in corpus]

embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2",
                               device="cuda")

# Change the max length to 512
embedder.max_seq_length = 512

In [ ]:
# Encode the sentences into embeddings
corpus_embeddings = embedder.encode(corpus_sentences,
                                    convert_to_numpy=True,
                                    show_progress_bar=True)

Batches:   0%|          | 0/258 [00:00<?, ?it/s]

In [ ]:
# Save corpus_embeddings to a file to be able to load later
import pickle

with open("corpus_embeddings_uk.pkl", "wb") as embeddings:
  pickle.dump({'corpus': corpus, 'embeddings': corpus_embeddings}, embeddings)

In [ ]:
# To load the embeddings later from the file instead of creating from scratch
import pickle

with open("corpus_embeddings_uk.pkl", "rb") as embeddings:
  data = pickle.load(embeddings)
  corpus = data['corpus']
  corpus_sentences = [item[0] for item in corpus]
  corpus_embeddings = data['embeddings']

In [ ]:
corpus_embeddings.shape

(8238, 384)

In [ ]:
import faiss

embedding_size = 384  # as in the model
n_clusters = 128
top_k_hits = 20

quantizer = faiss.IndexFlatL2(embedding_size)
index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters)

# Number of clusters to explorer at search time.
# We will search for nearest neighbors in 8 clusters
index.nprobe = 8

### Create the FAISS index
print("Start creating FAISS index")

# Train the index to find a suitable clustering
index.train(corpus_embeddings)

# Add all embeddings to the index
index.add(corpus_embeddings)

print("Number of embeddings indexed:", index.ntotal)

Start creating FAISS index
Number of embeddings indexed: 8238


In [ ]:
from sentence_transformers import SentenceTransformer

queries = ["education and schools in Germany"]
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

query_embeddings = model.encode(queries)

# Search in FAISS. It returns a matrix with distances and corpus ids.
distances, corpus_ids = index.search(query_embeddings, k=top_k_hits)

print(corpus_ids, "\n")

results = sorted([result for result in zip(distances.flatten(), corpus_ids.flatten())])
print(results, "\n")

for distance, idx in results:
  print(corpus_sentences[idx])
  print(f"Read more: {corpus[idx][1]} - {corpus[idx][2]}: {corpus[idx][3]}")
  print(f"Distance: {round(distance.item(), 2)}\n")

[[3946 3945 4077 6097 3947 8115 4052 3949 2064 6093 3939 6223 6277 8024
  4050 1080  643  857 1799 1624]] 

[(9.90493, 3946), (12.193857, 3945), (18.186539, 4077), (18.633497, 6097), (18.761229, 3947), (18.84115, 8115), (19.09331, 4052), (19.170555, 3949), (19.265438, 2064), (19.340284, 6093), (19.443645, 3939), (19.470036, 6223), (19.77827, 6277), (19.842066, 8024), (19.857483, 4050), (19.932281, 643), (19.932281, 857), (19.932281, 1080), (19.932281, 1624), (19.932281, 1799)] 

Як влаштувати дитину до школи в Німеччині? | Розповідь інсайдера |Відповіді на всі запитання |
Read more: Німеччина - Діти: https://uahelpinfo.notion.site/bb09aeaa08fa4e309e7fe8136427e686
Distance: 9.9

Після оформлення та отримання відповідних документів, виникнуть питання в яку школу віддавати дітей і яким чином тут це налаштовано. Більше інформації про систему освіти у Німеччині можна прочитати тут.
Read more: Німеччина - Діти: https://uahelpinfo.notion.site/bb09aeaa08fa4e309e7fe8136427e686
Distance: 12.19



In [ ]:
results[:5]

[(9.90493, 3946),
 (12.193857, 3945),
 (18.186539, 4077),
 (18.633497, 6097),
 (18.761229, 3947)]

In [ ]:
# Reranking input [(query, paragraph), (query, paragraph), (query, paragraph), ...]

reranker_input = [(queries[0], corpus[result[1]][0]) for result in results]
reranker_input[:5]

[('education and schools in Germany',
  'Як влаштувати дитину до школи в Німеччині? | Розповідь інсайдера |Відповіді на всі запитання |'),
 ('education and schools in Germany',
  'Після оформлення та отримання відповідних документів, виникнуть питання в яку школу віддавати дітей і яким чином тут це налаштовано. Більше інформації про систему освіти у Німеччині можна прочитати тут.'),
 ('education and schools in Germany',
  'Група вивчення німецької для переселенців. Курс для абсолютних початківців. Щочетверга о 10 годині віденського часу. https://t.me/deutschfueraliens_kurs'),
 ('education and schools in Germany',
  'Австрійські школи розглядають українських дітей як особливих учнів (не ставлять оцінки, домашнє завдання не дають), де основна увага у навчальному процесі приділяється саме мові та адаптації, а не фактичному навчанню, бо без мови – складно. Якщо ціль саме предмети та навчання, можно спробувати онлайн-уроки від Міносвіти'),
 ('education and schools in Germany',
  'Українська

In [18]:
# [Optional] Reranking
# After retrieving the top-k candidates, we can re-rank them with a cross-encoder model

from sentence_transformers import CrossEncoder

model = CrossEncoder("amberoad/bert-multilingual-passage-reranking-msmarco", max_length=512)

reranker_scores = model.predict(reranker_input)

# label 0: not relevant, and label 1: relevant
reranker_scores

array([[ 3.5964844 , -2.7340245 ],
       [ 0.9109123 , -0.46547103],
       [ 5.148391  , -4.193904  ],
       [ 4.597375  , -3.6226785 ],
       [ 4.9956627 , -4.025356  ],
       [ 5.5680523 , -4.7244687 ],
       [ 2.605761  , -1.737217  ],
       [ 3.3223052 , -2.395006  ],
       [ 4.561674  , -3.5567892 ],
       [ 4.5246496 , -3.4602792 ],
       [ 5.5370965 , -4.5752387 ],
       [ 5.113559  , -4.1691885 ],
       [ 5.7684865 , -4.8660135 ],
       [ 5.731864  , -4.7733564 ],
       [ 4.2481375 , -3.2506895 ],
       [ 5.56897   , -4.614217  ],
       [ 5.56897   , -4.614217  ],
       [ 5.56897   , -4.614217  ],
       [ 5.56897   , -4.614217  ],
       [ 5.56897   , -4.614217  ]], dtype=float32)

In [24]:
# [Optional] Convert logits to probabilities for readability or to apply a threshold

import numpy as np

# Convert logits to probabilities using the sigmoid function
reranker_scores = [1 / (1 + np.exp(-score)) for score in reranker_scores]
reranker_scores

[array([0.97331184, 0.06099525], dtype=float32),
 array([0.7131868 , 0.38568872], dtype=float32),
 array([0.99422485, 0.01486303], dtype=float32),
 array([0.99002224, 0.02601612], dtype=float32),
 array([0.9932782 , 0.01754379], dtype=float32),
 array([0.9961966 , 0.00879735], dtype=float32),
 array([0.93123144, 0.14966679], dtype=float32),
 array([0.9651861 , 0.08355431], dtype=float32),
 array([0.9896634 , 0.02773888], dtype=float32),
 array([0.9892777 , 0.03046378], dtype=float32),
 array([0.9960775 , 0.01019875], dtype=float32),
 array([0.99402136, 0.01522929], dtype=float32),
 array([0.99688524, 0.00764512], dtype=float32),
 array([0.9967694 , 0.00838113], dtype=float32),
 array([0.98591053, 0.03730212], dtype=float32),
 array([0.9962   , 0.0098127], dtype=float32),
 array([0.9962   , 0.0098127], dtype=float32),
 array([0.9962   , 0.0098127], dtype=float32),
 array([0.9962   , 0.0098127], dtype=float32),
 array([0.9962   , 0.0098127], dtype=float32)]

In [25]:
# full hits from the corpus with links
full_hits = [[result[0], corpus[result[1]]] for result in results]
reranker_output = zip(reranker_scores, full_hits)

# Compare the results before and after reranking
# for score, hit in zip(reranker_scores, full_hits):
#   print(score, hit)

sorted_reranked_output = sorted([(score[1], hit[1]) for score, hit in reranker_output], reverse=True)

for score, hit in sorted_reranked_output:
  print(f"{hit[0]} \nRead more: {hit[1]} - {hit[2]}: {hit[3]} \nScore: {round(score.item(), 2)}\n")

Після оформлення та отримання відповідних документів, виникнуть питання в яку школу віддавати дітей і яким чином тут це налаштовано. Більше інформації про систему освіти у Німеччині можна прочитати тут. 
Read more: Німеччина - Діти: https://uahelpinfo.notion.site/bb09aeaa08fa4e309e7fe8136427e686 
Score: 0.39

Університет Людвіга Максиміліян (ЛМУ) в Мюнхені збирає пожертви на підтримку студентів та професорів різними способами (разова фінансова підтримка / стипендії / допомога для боротьби з німецькою владою / консультуванням щодо німецьких класів та варіантів навчання в ЛМУ). 
Read more: Німеччина - Освіта та мовні курси: https://uahelpinfo.notion.site/d6214a719110464d88cb2609515d7a1e 
Score: 0.15

Зверніть увагу, що для відвідування школи в Німеччині дітям зазвичай потрібно довести, що вони вакциновані проти кору. Будь ласка, зверніться до місцевої влади, де ваші діти можуть зробити щеплення. 
Read more: Німеччина - Діти: https://uahelpinfo.notion.site/bb09aeaa08fa4e309e7fe8136427e686

# GPU

In [ ]:
# To load the embeddings later from the file instead of creating from scratch
import pickle

with open("corpus_embeddings_uk.pkl", "rb") as embeddings:
    data = pickle.load(embeddings)
    corpus = data['corpus']
    corpus_sentences = [item[0] for item in corpus]
    corpus_embeddings = data['embeddings']

In [ ]:
import os

# Which GPU to use (if you have multiple GPUs)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # or "0,1" for multiple gpus

# For debugging CUDA errors
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [ ]:
!echo $CUDA_VISIBLE_DEVICES

In [ ]:
# Single GPU

import faiss

embedding_size = 384
n_clusters = 16
top_k_hits = 10

quantizer = faiss.IndexFlatL2(embedding_size)
index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters, faiss.METRIC_L2)

# Number of clusters to explorer at search time.
# We will search for nearest neighbors in 8 clusters
index.nprobe = 8

ngpus = faiss.get_num_gpus()
print("Number of GPUs:", ngpus)

res = faiss.StandardGpuResources()  # use a single GPU
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index)
print("Now the index in on GPU.")

# Train the index to find a suitable clustering
assert not gpu_index_flat.is_trained
gpu_index_flat.train(corpus_embeddings)
assert gpu_index_flat.is_trained
print("Training complete!")

gpu_index_flat.add(corpus_embeddings)  # add vectors to the index
print(gpu_index_flat.ntotal, "added.")

In [ ]:
# Multiple GPUs

import faiss

embedding_size = 384
n_clusters = 64
top_k_hits = 10


quantizer = faiss.IndexFlatL2(embedding_size)
index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters)

# Number of clusters to explorer at search time.
# We will search for nearest neighbors in 8 clusters
index.nprobe = 8


print("Moving index to gpu before training")

ngpus = faiss.get_num_gpus()
print("Number of GPUs:", ngpus)

gpu_index_flat = faiss.index_cpu_to_all_gpus(index)
print("Now the index in on GPU.")

# Train the index to find a suitable clustering
assert not gpu_index_flat.is_trained
gpu_index_flat.train(corpus_embeddings)
assert gpu_index_flat.is_trained
print("Training complete!")

# Add vectors to the index
gpu_index_flat.add(corpus_embeddings)
print(gpu_index_flat.ntotal, "added.")